In [7]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = ''
import random

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from omegaconf import OmegaConf
from hydra import compose, initialize
from omegaconf import OmegaConf
from hydra.utils import instantiate
from torchinfo import summary
%load_ext autoreload
%autoreload 2

from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path = "../training/conf/network/vqvae.yaml"

In [5]:
with initialize(config_path="../training/conf/network/", job_name="test_app"):
    cfg = compose(config_name="vqvae")
    print(OmegaConf.to_yaml(cfg))

encoder:
  _target_: text_recognizer.networks.vqvae.encoder.Encoder
  in_channels: 1
  hidden_dim: 32
  channels_multipliers:
  - 1
  - 2
  - 4
  dropout_rate: 0.0
  activation: mish
  use_norm: true
  num_residuals: 4
  residual_channels: 32
decoder:
  _target_: text_recognizer.networks.vqvae.decoder.Decoder
  out_channels: 1
  hidden_dim: 32
  channels_multipliers:
  - 4
  - 2
  - 1
  dropout_rate: 0.0
  activation: mish
  use_norm: true
  num_residuals: 4
  residual_channels: 32
_target_: text_recognizer.networks.vqvae.vqvae.VQVAE
hidden_dim: 128
embedding_dim: 32
num_embeddings: 8192
decay: 0.99



/home/aktersnurra/.cache/pypoetry/virtualenvs/text-recognizer-ejNaVa9M-py3.9/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'vqvae': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [6]:
net = instantiate(cfg)

In [9]:
summary(net, (1, 1, 576, 640), device="cpu")

Layer (type:depth-idx)                             Output Shape              Param #
VQVAE                                              --                        --
├─Encoder: 1-1                                     [1, 128, 72, 80]          --
│    └─Sequential: 2-1                             [1, 128, 72, 80]          --
│    │    └─Conv2d: 3-1                            [1, 32, 576, 640]         320
│    │    └─Normalize: 3-2                         [1, 32, 576, 640]         64
│    │    └─Mish: 3-3                              [1, 32, 576, 640]         --
│    │    └─Mish: 3-4                              [1, 32, 576, 640]         --
│    │    └─Mish: 3-5                              [1, 32, 576, 640]         --
│    │    └─Conv2d: 3-6                            [1, 32, 288, 320]         16,416
│    │    └─Normalize: 3-7                         [1, 32, 288, 320]         64
│    │    └─Mish: 3-8                              [1, 32, 288, 320]         --
│    │    └─Mish: 3-9         